In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

global_seed = 2022

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
!pip install datasets

import datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.8/346.8 KB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 9.4 MB/s eta 0:00:00


# Dataset

https://huggingface.co/datasets/ucberkeley-dlab/measuring-hate-speech

In [3]:
dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech', 'utf-8')   
dataset = dataset['train'].to_pandas()
dataset.head(1)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/ucberkeley-dlab___parquet/ucberkeley-dlab--measuring-hate-speech-f91f636a830ad73c/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

,comment_id,annotator_id,platform,sentiment,respect,insult,humiliate,status,dehumanize,violence,genocide,attack_defend,hatespeech,hate_speech_score,text,infitms,outfitms,annotator_severity,std_err,annotator_infitms,annotator_outfitms,hypothesis,target_race_asian,target_race_black,target_race_latinx,target_race_middle_eastern,target_race_native_american,target_race_pacific_islander,target_race_white,target_race_other,target_race,target_religion_atheist,target_religion_buddhist,target_religion_christian,target_religion_hindu,target_religion_jewish,target_religion_mormon,target_religion_muslim,target_religion_other,target_religion,target_origin_immigrant,target_origin_migrant_worker,target_origin_specific_country,target_origin_undocumented,target_origin_other,target_origin,target_gender_men,target_gender_non_binary,target_gender_transgender_men,target_gender_transgender_unspecified,target_gender_transgender_women,target_gender_women,target_gender_other,target_gender,target_sexuality_bisexual,target_sexuality_gay,target_sexuality_lesbian,target_sexuality_straight,target_sexuality_other,target_sexuality,target_age_children,target_age_teenagers,target_age_young_adults,target_age_middle_aged,target_age_seniors,target_age_other,target_age,target_disability_physical,target_disability_cognitive,target_disability_neurological,target_disability_visually_impaired,target_disability_hearing_impaired,target_disability_unspecific,target_disability_other,target_disability,annotator_gender,annotator_trans,annotator_educ,annotator_income,annotator_ideology,annotator_gender_men,annotator_gender_women,annotator_gender_non_binary,annotator_gender_prefer_not_to_say,annotator_gender_self_describe,annotator_transgender,annotator_cisgender,annotator_transgender_prefer_not_to_say,annotator_education_some_high_school,annotator_education_high_school_grad,annotator_education_some_college,annotator_education_college_grad_aa,annotator_education_college_grad_ba,annotator_education_professional_degree,annotator_education_masters,annotator_education_phd,annotator_income_<10k,annotator_income_10k-50k,annotator_income_50k-100k,annotator_income_100k-200k,annotator_income_>200k,annotator_ideology_extremeley_conservative,annotator_ideology_conservative,annotator_ideology_slightly_conservative,annotator_ideology_neutral,annotator_ideology_slightly_liberal,annotator_ideology_liberal,annotator_ideology_extremeley_liberal,annotator_ideology_no_opinion,annotator_race_asian,annotator_race_black,annotator_race_latinx,annotator_race_middle_eastern,annotator_race_native_american,annotator_race_pacific_islander,annotator_race_white,annotator_race_other,annotator_age,annotator_religion_atheist,annotator_religion_buddhist,annotator_religion_christian,annotator_religion_hindu,annotator_religion_jewish,annotator_religion_mormon,annotator_religion_muslim,annotator_religion_nothing,annotator_religion_other,annotator_sexuality_bisexual,annotator_sexuality_gay,annotator_sexuality_straight,annotator_sexuality_other
0,47777,10873,3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,-3.9,"Yes indeed. She sort of reminds me of the elder lady that played the part in the movie ""Titanic"" who was telling her story!!! And I wouldn't have wanted to cover who I really am!! I would be proud!!!! WE should be proud of our race no matter what it is!!",0.81,1.88,0.36,0.34,1.35,1.23,-1.130178,True,True,True,True,True,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,male,no,college_grad_ba,<10k,neutral,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,25.0,False,False,True,False,False,False,False,False,False,False,Fa

The hatespeech column has three values
* 0 for positive comments
* 1 when not clear
* 2 for hate speech

This value can differ among annotators for the same comment_id. But the calculated hate_speech_score will be the same. 

For simplicity in experiment we change the dataset into binary classification by removing examples where annotators are not clear.

In [4]:
text_column = 'text'
target_column = 'labels'


# https://stackoverflow.com/questions/8689795/how-can-i-remove-non-ascii-characters-but-leave-periods-and-spaces
# Make sure all comment_text values are strings
dataset.loc[:, text_column] = dataset[text_column].astype(str) 
dataset.loc[:, target_column] = dataset['hatespeech'].astype(int)
dataset = dataset[dataset[target_column] != 1]
dataset.loc[:, target_column]  =dataset[target_column].map({0:0, 2:1})

## Drop unnecessary columns and duplicates

In [5]:
id_columns = ['comment_id']
features = [text_column, target_column]
selected_columns = id_columns + features

dataset = dataset[selected_columns]

## Aggregation

Dataset contains multiple entries for each comment id from different annotators.

In [6]:
%%time
# https://stackoverflow.com/questions/15222754/groupby-pandas-dataframe-and-select-most-common-value
grouped = dataset.groupby(id_columns)[[target_column]].agg(lambda x: pd.Series.mode(x)[0]).reset_index()
dataset_unique = dataset.drop_duplicates(subset=id_columns)

CPU times: user 4.76 s, sys: 28.8 ms, total: 4.79 s
Wall time: 4.78 s


In [7]:
merged = grouped.merge(dataset_unique[id_columns+[text_column]], on=id_columns, how='inner').reset_index(drop=True)
merged

,comment_id,labels,text
0,1,0,White people are trash.
1,2,1,"You've caught the big gay, it's known to be airborne via dildos."
2,3,1,"I'm not saying you said that, I'm just saying that if you find a slur offensive (in this case the word niga) you'd find every other slur offensive. For me it's the opposite, I find none of them offensive because unless the context is bad no one should give a shit. I can call my black friend ""niga"" in a friendly way or in an evil way. There's a difference"
3,4,1,"Donald Trump. Yeet myself off a building onto a gay dudes cock. And by gay dude, I mean Mike Pence."
4,5,0,"Fabrice Fabrice is ostensibly black or black/latino (URL so that's part of it, beyond the gay stereotypes."
...,...,...,...
38559,50066,1,Good you Americans are annoying asf. I Wish more Americans would die by the PHONE today
38560,50067,1,"Go to streets of the USA and our kids and vets and build the wall , use your brain you b...."
38561,50068,0,That Kenyan girl is gorgeous!
38562,50069,0,"Yikes!!! Bad enough she'shard to look at.everything comes out her mouth is bullshit. That's why i fear the dems are gonna loose BIGLY ,loudmouth nothing burger."


# Length distribution

In [8]:
lengths = []
for text in merged[text_column].values:
    lengths.append(len(text.split(' ')))
    
lengths = pd.DataFrame(lengths)
lengths.describe()

,0
count,38564.000000
mean,25.286822
std,19.819185
min,1.000000
25%,11.000000
50%,19.000000
75%,35.000000
max,128.000000


# Train-test split

In [9]:
merged[target_column].value_counts()

0    29518
1     9046
Name: labels, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(
    merged.drop(columns=target_column),
    merged[target_column],
    test_size=0.3,
    random_state=global_seed
)

x_val, x_test, y_val, y_test = train_test_split(
    x_val,
    y_val,
    test_size=0.5,
    random_state=global_seed
)

## Value counts

In [11]:
def value_count(df, value):
    counts = df[value].value_counts().reset_index()
    counts.columns = ['Value', 'Count']
    counts['Count(%)'] = counts['Count'] * 100 / counts['Count'].sum()
    print(counts, '\n')

In [12]:
x_train[target_column] = y_train
x_val[target_column] = y_val
x_test[target_column] = y_test

train_df = x_train.reset_index(drop=True)
val_df = x_val.reset_index(drop=True)
test_df = x_test.reset_index(drop=True)

In [13]:
print('Total dataset')
value_count(merged, target_column)

print('Train dataset')
value_count(train_df, target_column)

print('Validation dataset')
value_count(val_df, target_column)

print('Test dataset')
value_count(test_df, target_column)

Total dataset
   Value  Count  Count(%)
0      0  29518  76.54289
1      1   9046  23.45711 

Train dataset
   Value  Count   Count(%)
0      0  20655  76.517004
1      1   6339  23.482996 

Validation dataset
   Value  Count  Count(%)
0      0   4411  76.24892
1      1   1374  23.75108 

Test dataset
   Value  Count   Count(%)
0      0   4452  76.957649
1      1   1333  23.042351 



## Convert to dataset

In [14]:
train_dataset = datasets.Dataset.from_pandas(train_df)
val_dataset = datasets.Dataset.from_pandas(val_df)
test_dataset = datasets.Dataset.from_pandas(test_df)

# Tokenize

https://huggingface.co/docs/transformers/main_classes/tokenizer

In [15]:
from transformers import AutoTokenizer

model_name = "bert-base-uncased"
# model_name = 'prajjwal1/bert-small'
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    do_lower_case=True,
)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [16]:
MAX_SEQ_LENGTH = 128

def tokenize_function(examples):
    return tokenizer(list(examples[text_column]), padding="max_length", max_length=MAX_SEQ_LENGTH, truncation=True)

In [17]:
train_tokenized = train_dataset.map(tokenize_function, batched=True)
val_tokenized = val_dataset.map(tokenize_function, batched=True)
test_tokenized = test_dataset.map(tokenize_function, batched=True)

  0%|          | 0/27 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [18]:
train_tokenized.column_names, val_tokenized.column_names, test_tokenized.column_names

(['comment_id',
  'text',
  'labels',
  'input_ids',
  'token_type_ids',
  'attention_mask'],
 ['comment_id',
  'text',
  'labels',
  'input_ids',
  'token_type_ids',
  'attention_mask'],
 ['comment_id',
  'text',
  'labels',
  'input_ids',
  'token_type_ids',
  'attention_mask'])

In [19]:
# https://huggingface.co/docs/datasets/access
# drop string columns because they cause error during training phase

train_tokenized = train_tokenized.remove_columns([text_column])
train_tokenized.set_format("torch")

val_tokenized = val_tokenized.remove_columns([text_column])
val_tokenized.set_format("torch")

test_tokenized = test_tokenized.remove_columns([text_column])
test_tokenized.set_format("torch")

## Dump tokenized data

In [20]:
import pickle

with open('train.pkl', 'wb') as output:
    pickle.dump(train_tokenized, output, pickle.HIGHEST_PROTOCOL)
    output.close()
    
with open('validation.pkl', 'wb') as output:
    pickle.dump(val_tokenized, output, pickle.HIGHEST_PROTOCOL)
    output.close()
    
with open('test.pkl', 'wb') as output:
    pickle.dump(test_tokenized, output, pickle.HIGHEST_PROTOCOL)
    output.close()